# Imitaion learning paper review
&nbsp;&nbsp;&nbsp;  **Main** : A Reduction of Imitation learning and Structured Prediction to No-regret Online learning  
&nbsp;&nbsp; &nbsp;  **Sub**  : Efficient Reductions for Imitation learning

## 0.Background
Imitation learning은 그 동안 supervised learning problem로 생각하고 풀어왔다. 하지만, 이는 각각의 training set과 test set 들이 i.i.d(identical independent  distribution) 조건을 따른다고 가정했을 때 supervised learning problem으로 생각하고 풀 수 있지만, 실제로 test input들로 주어지는 것들을 현재 policy에 depencent 하게 바뀌기 때문에 불가능하다.

## 1.Problem setting

$\pi^*$ : the policy of expert  (deterministic)  
$\pi_s$ : policy에 따른 state $s$에 대한 actions 의 distributions

### 1.1 cost from actual implementation
$C(s,a) \in [0,1]$ : state $s$에서 action $a$를 수행했을 때 생긴 immediate cost.  
$C_{\pi}(s) = E_{a\sim \pi_s}[C(s,a)]$ : state $s$ 에서 policy $\pi$로 수행했을 때 생긴 immediate cost의 expectation value.


### 1.2 Error between expert policy
$e(s,a) = I(a\neq \pi^{*}(s)) $ expert의 policy $\pi^{*}$이랑 다를 경우 1, 같을 경우 0

$e_{\pi}(s) = E_{a\sim \pi_s}[e(s,a)]$ policy $\pi$를 택할 경우 state $s$에서의 $e(s,a)$의 expectation value

### 1.3 State들의 distribution
전체 Time $T$ 까지 학습한다고 가정  
모든 Policy들의 집합 $\Pi$  
$d_{\pi}^{i} : $ policy $\pi$를 따른다고 했을 때 time step $i$에서 도달할 state 들의 probabilistic 한 distribution  
$d_{\pi} = \frac{1}{T} \sum_{i=1}^{T} d_{\pi}^i : $ 전체 time step에서 각 state 들의 probabilistic 한 distribution(각 state 들을 들린 frequency 로도 생각할 수 있다.)

### 1.4 Total cost
**policy $\pi$를 택할 경우 total cost**
$$J(\pi) = \sum_{i=1}^T E_{s\sim d_{\pi}^i}[C_{\pi}(s))]
=TE_{s\sim d_{\pi}}[C_{\pi}(s)]$$ 


### 1.5 Regret(in total cost)

Policy 집합 $\Pi$에서 제일 좋은 policy와 cost 차이를 regret이라고 한다. 수식으로 적어보면
$$\mathcal{R}_{\Pi}(\pi) = J(\pi)- \min_{\pi'\in \Pi} J(\pi ')$$

Expert의 policy $\pi^*$ 가 좋은 기준이 되려면 
$$\mathcal{R}_{\Pi}(\pi^*)$$ 가 O(1)이어야 한다.

## 2. Preliminaries

### 2.1 Supervised Approach to Imitation
이게 왜 문제인지를 깨닫는 것이 중요하다!
$$\hat{\pi} = \arg \min_{\pi \in \Pi} E_{s\sim d_{\pi^*}}[e_{\pi}(s) ]$$

$\Rightarrow$expert policy $\pi^*$를 실행했을 때 생길 수 있는 state들에 대해서 $\pi^*$ 와 가장 비슷하게 동작하는 $\pi$를 찾는다.
하지만, 실제로는 $\hat{\pi}$ 는 다른 state 들의 distribution에 대해서 실행되게 된다.

#### Theorem 2.1.1
Let $E_{s\sim d_{\pi^*}}[e_{\hat{\pi}}(s)] = \epsilon$, then 
$J(\hat{\pi}) \leq J(\pi^*)+T^2\epsilon$  
  
* 전체 Cost의 upper bound가 $T^2$ 에 비례해서 증가한다.
* 그리고 실제로 그 전 논문에서 저 upper bound에 가깝게 작동하는 예시를 들어주었다.

### 2.2 Forward Training

이번에는 각각에 time $t \in [1,2,\cdots, T]$ 에 대해서 서로 다른 policy $\pi_t$ 를 각각 학습시킨다.  

*Algorithm*  
(note $\pi_j^i $ 는 for loop $i$ 번째에서 학습된 time step $j$ 에서의 policy)  

>$\pi_1^{0}, \cdots, \pi_T^0$ 초기화  
>**for** $i$ in $[1, \cdots, T]$   
>$\hspace{0.4cm}$ policy $\pi^{i-1}$s 를 통해 $T$-step 시행을 여러번 하여 sampling을 한다.   
>$\hspace{0.4cm}$ Time step $i$에서 얻어진 $s_i$ 와 이에 따른 expert의 action $\pi^*(s_i)$을 통해 $D = \{(s_i, \pi^{*}(s_i))\}$을 만든다.
>$\hspace{0.4cm}$ $\pi_i^{i} = \arg\min_{\pi \in \Pi} E_{s\sim D}[e_{\pi}(s)]$ 을 학습시켜서 얻는다.(아마 방법은 자유)   
>$\hspace{0.4cm}$ $\pi_j^{i} = \pi_j^{i-1}$ for $j\neq i $   
>**Return** $\pi_1^{T}, \cdots, \pi_T^T$ 


$Q_t^{\pi'}(s,\pi) : $ 처음 state $s$ 에서 policy $\pi$를 실행하고 그 이후에는 policy $\pi^*$을 실행했을 때 생긴 $t$ step에서의 cost

#### Theorem 2.2.1
생략.. 결론은 total cost의 upper bound가 $T$에 비례하게 증가하게 된다.

#### Drawback
1. $T$가 너무 커지면 계산량이 많아진다.
2. policy가 non-stationary policy이고 그렇기에 무한대로 확장 불가능하다.

### 2.3 Stochastic Mixing Iterative Learning

*Algorithm*    

>$\pi^{0} \leftarrow \pi^*$ 초기화
>>사실 expert policy를 넣는 느낌이 아니라 $\pi^{0}$ 은 expert 한테 물어본 후 그 결과 가져다 쓴다는 느낌

>**for** $i$ in $[1, \cdots, N]$   
>$\hspace{0.4cm}$ policy $\pi^{i-1}$ 를 통해 state $s$ 를 얻고 거기에 따른$D = \{(s, \pi^{*}(s))\}$을 만든다.  
>$\hspace{0.4cm}$ $\hat{\pi}_i = \arg\min_{\pi \in \Pi} E_{s\sim D}[e_{\pi}(s)]$ 을 학습시켜서 얻는다.(아마 방법은 자유)   
>$\hspace{0.4cm}$ $\pi^{i} = \pi^{i-1}+\alpha(1-\alpha)^{i-1}(\hat{\pi}_i - \pi^*)$  
>> 학습된 $\hat{\pi}_i$ 의 특성을 넣고 expert에 의한 영향 $\pi^*$을 제거하면서 학습된다. 위 식은 사실 다음과 동치
>> $\pi^{i} = (1-\alpha)^i\pi^*+ \alpha\sum_{j=1}^{i}(1-\alpha)^{j-1}\hat{\pi}_j$  
>> 이 식으로 해석하면 학습된 $i$개의 policy를 갖다쓰면서도 $(1-\alpha)^i$ 만큼은 expert의 도움을 받는다고 볼 수 있다.
  
>**Return** $\frac{\pi^N-(1-\alpha)^N\pi^*)}{1-(1-\alpha)^N}$ 
>> expert의 영향만큼을 빼주고 normalize 한 것과 동일 

## 3. DAGGER(dataset aggregation)

*Algorithm*    

>$D=\emptyset$  
>$\hat{\pi}_{1} :  \Pi$ 중 임의의 policy로 초기화  

>**for** $i$ in $[1, \cdots, N]$   
>$\hspace{0.4cm}$ $\pi_i = \beta_i \pi^*+(1-\beta_i) \hat{\pi}_i$   
>$\hspace{0.4cm}$ $\pi_i$를 이용해 $T$-step 시행을 여러번 하여 sampling  
>$\hspace{0.4cm}$ 위의 sampling 결과에서 state $s$를 얻고, 그에 따른 expert 의 policy $\pi^*$를 통해 $D_i = \{(s,\pi^*(s))\}$ 를 만든다.  
>$\hspace{0.4cm}$ $D=D\cup D_i$  
>$\hspace{0.4cm}$  $\hat{\pi}_{i+1} = \arg\min_{\pi \in \Pi} E_{s\sim D}[e_{\pi}(s)]$ 을 학습시켜서 얻는다. (아마 방법은 error function을 다른 것으로 바꿀 수 있을 것으로 보인다.)   
>**Return** validation set에서 가장 잘 동작하는 $\hat{\pi_i}$  

$\beta_1 =1$, and $$\lim_{N\rightarrow \infty}\frac{1}{N}\sum_{i=1}^{N} \beta_i = 0$$  
Ex) $\beta_i = p^{i-1} $

*DAGGER*은 매 iteration 마다 expert의 policy와 합쳐진 policy를 이용해 dataset을 축적하고, 그 데이터를 바탕으로 policy를 update 한다.
dataset에는 해당 policy에 의해 도달할 거 같은 state들과 그것의 expert 시행들이 축적되게 된다. 

## 5. Experiment

Dagger > SMIL > supervised